In [1]:
import numpy as np
import scipy.stats as sps

In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

tf.executing_eagerly()

2021-08-31 12:52:14.001997: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


True

In [3]:
# thanks https://stackoverflow.com/questions/34199233/how-to-prevent-tensorflow-from-allocating-the-totality-of-a-gpu-memory
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

2021-08-31 12:52:14.931864: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-08-31 12:52:14.956644: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 12:52:14.956925: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 with Max-Q Design computeCapability: 7.5
coreClock: 1.095GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 357.69GiB/s
2021-08-31 12:52:14.956942: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-08-31 12:52:14.958055: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-08-31 12:52:14.959156: I tensorflow/s

In [4]:
Pa = np.array([[0.71893144, 0.14144765, 0.10698649, 0.03263443],
       [0.06325843, 0.77423272, 0.13928361, 0.02322523],
       [0.02322523, 0.13928361, 0.77423272, 0.06325843],
       [0.03263443, 0.10698649, 0.14144765, 0.71893144]])

Pb = np.array([[0.95154866, 0.01674797, 0.0234783 , 0.00822507],
       [0.03863562, 0.84803839, 0.04732459, 0.0660014 ],
       [0.0660014 , 0.04732459, 0.84803839, 0.03863562],
       [0.00822507, 0.0234783 , 0.01674797, 0.95154866]])

Pc = np.array([[0.88137945, 0.04613594, 0.02686734, 0.04561727],
       [0.05009116, 0.9236776 , 0.0194023 , 0.00682894],
       [0.00682894, 0.0194023 , 0.9236776 , 0.05009116],
       [0.04561727, 0.02686734, 0.04613594, 0.88137945]])
Pm = np.array([[0.81203158, 0.04399917, 0.07848442, 0.06548484],
       [0.08284662, 0.87115553, 0.02024715, 0.0257507 ],
       [0.0257507 , 0.02024715, 0.87115553, 0.08284662],
       [0.06548484, 0.07848442, 0.04399917, 0.81203158]])

pi = np.array([0.14977394, 0.26317362, 0.39165429, 0.19539816])

pi_m = pi @ Pm

J = np.array([[[0.07508613297288518, 0.00466350788803318, 0.0033432901131033796, 0.004042219322757277], [0.002186937387234338, 0.01570096614179908, 0.0011254388766303509, 0.0005015574893929194], [0.002007283482847851, 0.0012535337368050612, 0.013395860029779497, 0.0010220788989917808], [0.0013011086711092986, 0.0016042909684947522, 0.0012404869820179004, 0.011323417297654589]], [[0.03129698648252512, 0.00815953485072002, 0.004394876926278819, 0.0020308813605868174], [0.008288184321098279, 0.14201837079233715, 0.005398759767993272, 0.0017880581744532536], [0.0015277493099248596, 0.00886482820182737, 0.04269081732021271, 0.0028169181161841477], [0.0021015052010260205, 0.011819352252411985, 0.0034103593745217937, 0.02379160092199082]], [[0.02174460265532043, 0.002941403304710777, 0.017006110729978843, 0.0023681733984058486], [0.0022577498981608663, 0.03237071808690972, 0.012426756488990255, 0.0019160961759307586], [0.0022144999652547693, 0.006239396030284971, 0.20953281339937846, 0.012130269098422072], [0.002543170970124578, 0.003973434481032667, 0.011775618913078878, 0.04210892174683528]], [[0.012093125924573445, 0.0012993605630256348, 0.0021464967072447926, 0.0015645368675293004], [0.0010702747292501203, 0.013479197587703494, 0.0016493822206027895, 0.0025896804160072057], [0.0005894417935184289, 0.00127918145334602, 0.022260289874459077, 0.0029430040436876307], [0.005229012301832341, 0.0040480932067737535, 0.006152859243062461, 0.09785943409093556]]])

In [5]:
J_mb = pi_m[:, np.newaxis] * Pb
J_bc = J_mb.T @ Pc
J_ab = Pa.T @ np.diag(pi) @ Pm @ Pb
J_ca = (Pa.T @ np.diag(pi) @ Pm @ Pc).T

assert np.allclose(J_bc, J.sum(axis=0))  # checking I have constructed it correctly
assert np.allclose(J_ab, J.sum(axis=2))
assert np.allclose(J_ca, J.sum(axis=1).T)

J_einsum = np.einsum('i,ij,ik,ku,kv', pi, Pa, Pm, Pb, Pc)
assert np.allclose(J_einsum, J)

In [14]:
def sample(N, J):
    return sps.multinomial.rvs(N, J.flatten()).reshape((4, 4, 4))/N

In [6]:
# thanks https://github.com/mlgxmez/thelongrun_notebooks/blob/master/MLE_tutorial.ipynb

@tf.function()
def transform_P_matrix(params):
    params = tf.exp(params)
    Q0 = tf.concat([[-tf.reduce_sum(params[0])], params[0]],
                   axis=0)
    Q1 = tf.concat([[params[1,0]], [-tf.reduce_sum(params[1])], params[1,1:]],
                   axis=0)
    Q = tf.concat([[Q0], [Q1], [Q1[::-1]], [Q0[::-1]]], axis=0)
    return tf.linalg.expm(Q)

@tf.function()
def transform(params):
    pi = tfb.SoftmaxCentered()(params[0])
    Pa = transform_P_matrix(params[1:3])
    Pm = transform_P_matrix(params[3:5])
    Pb = transform_P_matrix(params[5:7])
    Pc = transform_P_matrix(params[7:9])
    return pi, Pa, Pm, Pb, Pc
    
@tf.function()
def _loss(params_data):
    params, data = params_data
    pi, Pa, Pm, Pb, Pc = transform(params)
    J = tf.einsum('i,ij,ik,ku,kv', pi, Pa, Pm, Pb, Pc)
    loss = tf.reduce_sum(tf.keras.losses.KLDivergence()(J, data))
    return loss

@tf.function()
def loss(params, data):
    return tf.reduce_sum(tf.vectorized_map(_loss, (parameters, data)))

@tf.function()
def training_step(parameters, data, optimizer):
    with tf.GradientTape() as tape:
        loss_value = loss(parameters, data)
    gradients = tape.gradient(loss_value, parameters)
    return loss_value, gradients

def mle_run(data, loss, parameters, optimizer, steps=500, verbose=False):
    
    for i in range(steps):
        loss_value, gradients = training_step(parameters, data, optimizer)
        optimizer.apply_gradients([(gradients, parameters)])
        
        if i % 100 == 0:
            iter_info = f"Step: {optimizer.iterations.numpy()}, initial loss: {loss_value.numpy()}"
            if verbose:
                print(iter_info)
    
    return parameters

In [25]:
%%time
N = 10000
K = 3
normal_initializer = tf.random_normal_initializer()
parameters = tf.Variable(normal_initializer(shape=[K, 9, 3], dtype=tf.float32), name='params')
Ja = sample(N, J).astype(np.float32)
Jb = Ja.transpose([1, 2, 0])
Jc = Ja.transpose([2, 0, 1])
data = tf.stack([Ja, Jb, Jc])

optimizer = tf.optimizers.Adam(0.01)
update_list = mle_run(data, loss, parameters, optimizer, steps=10000, verbose=True)

Step: 1, initial loss: 0.20222054421901703
Step: 101, initial loss: 0.16597172617912292
Step: 201, initial loss: 0.09657593071460724
Step: 301, initial loss: 0.07258816063404083
Step: 401, initial loss: 0.05790988355875015
Step: 501, initial loss: 0.050487734377384186
Step: 601, initial loss: 0.030205361545085907
Step: 701, initial loss: 0.006704481318593025
Step: 801, initial loss: 0.0014757821336388588
Step: 901, initial loss: 0.0007248119218274951
Step: 1001, initial loss: 0.0006472094682976604
Step: 1101, initial loss: 0.000616406206972897
Step: 1201, initial loss: 0.0005991252255626023
Step: 1301, initial loss: 0.0005879321834072471
Step: 1401, initial loss: 0.0005801510415039957
Step: 1501, initial loss: 0.000574370613321662
Step: 1601, initial loss: 0.0005698658060282469
Step: 1701, initial loss: 0.000566203729249537
Step: 1801, initial loss: 0.0005631373496726155
Step: 1901, initial loss: 0.0005604737671092153
Step: 2001, initial loss: 0.0005581151926890016
Step: 2101, initial 

In [26]:
for p, phat in zip([pi, Pa, Pm, Pb, Pc], transform(parameters[0])):
    print(p)
    print(phat.numpy())
    print()

[0.14977394 0.26317362 0.39165429 0.19539816]
[0.12204078 0.28609538 0.433098   0.15876588]

[[0.71893144 0.14144765 0.10698649 0.03263443]
 [0.06325843 0.77423272 0.13928361 0.02322523]
 [0.02322523 0.13928361 0.77423272 0.06325843]
 [0.03263443 0.10698649 0.14144765 0.71893144]]
[[0.8098485  0.0768582  0.06198487 0.0513085 ]
 [0.07545466 0.7581055  0.13507761 0.03136224]
 [0.03136223 0.13507761 0.75810546 0.07545465]
 [0.0513085  0.06198487 0.07685821 0.80984855]]

[[0.81203158 0.04399917 0.07848442 0.06548484]
 [0.08284662 0.87115553 0.02024715 0.0257507 ]
 [0.0257507  0.02024715 0.87115553 0.08284662]
 [0.06548484 0.07848442 0.04399917 0.81203158]]
[[0.8716503  0.00613635 0.0736386  0.04857485]
 [0.11426511 0.81730264 0.02106063 0.0473716 ]
 [0.0473716  0.02106064 0.81730264 0.11426511]
 [0.04857485 0.0736386  0.00613635 0.8716503 ]]

[[0.95154866 0.01674797 0.0234783  0.00822507]
 [0.03863562 0.84803839 0.04732459 0.0660014 ]
 [0.0660014  0.04732459 0.84803839 0.03863562]
 [0.0082

In [27]:
ls =  np.array([_loss((parameters[i], data[i])).numpy() for i in range(K)])
print(ls)
ls = N*ls
delta = ls - ls.min()
weights = np.exp(-delta)
weights /= weights.sum()
# ls /= ls.sum()
print(weights)

[0.00012748 0.00020096 0.00019958]
[0.5086934  0.24395621 0.24735038]
